# T3 de Inteligência Artificial

Aluno: Leonardo Souza

Professor: Eduardo Bezerra

O objetivo deste trabalho é a implementação do algoritmo Q-Learning pelo método de aproximação linear.

O algoritmo será testado dentro do ambiente Taxi-v3 do OpenGym, para tal, será necessário instalar algumas dependências:

    pip3 install gym==0.17.3



### Testando a criação do ambiente Taxi-v3

In [2]:
import gym

env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [61]:
import numpy as np
import torch


In [52]:
state = env.reset()
action = env.action_space.sample() # Seleciona ação a executar
new_state, reward, done, info = env.step(action)
print(env.action_space.n)

print(state)
print(action)
print(new_state, reward, done, info)


def feature_manhattan_distance_taxi_passenger(state, action):
    l, c, p, d = env.unwrapped.decode(state)
    p = env.unwrapped.locs[p]
    
    return abs(l - p[0]) + abs(c - p[1])


def feature_manhattan_distance_taxi_destiny(state, action):
    l, c, p, d = env.unwrapped.decode(state)
    d = env.unwrapped.locs[d]

    return abs(l - d[0]) + abs(c - d[1])


6
183
0
283 -1 False {'prob': 1.0}


In [79]:
# print(feature_manhattan_distance_taxi_passenger(state, action))
# print(feature_manhattan_distance_taxi_passenger(new_state, action))

# a = np.array([feature_manhattan_distance_taxi_passenger, feature_manhattan_distance_taxi_destiny])
b = np.random.rand(a.size)
print(b)
features = [f(state, action) for f in  [feature_manhattan_distance_taxi_passenger, feature_manhattan_distance_taxi_destiny]]
# features = np.array([f(state, action) for f in  [feature_manhattan_distance_taxi_passenger, feature_manhattan_distance_taxi_destiny]])
print(features)
features = np.array([5, 4])
print(type(features))

print(features * b)
print((features * b).sum())
print( b @ features)

[0.67955667 0.74524331]
[5, 4]
<class 'numpy.ndarray'>
[3.39778335 2.98097324]
6.378756591130507
6.378756591130507


## Implementação do Q-Learning Linear

### Definição dos hyperparâmetros

### Implementação das Classe QLearningLinear

In [ ]:
# Class QLearningLinear
class QLearningLinear(object):
    
    def __init__(self, learning_rate = 0.7, features:list):
        # cria um numpy array de pesos sendo 1:n, onde n representa o número de features
        self.weigths = np.random.rand(len(features))
        # covnerte a lista de features para um numpy array.
        self.features = features
    
    def Q(state, action):
        """
            Q(S,A) 
        """
        f_values = [f(state, action) for f in  [feature_manhattan_distance_taxi_passenger, feature_manhattan_distance_taxi_destiny]]
        f_values = np.array(f_values)

        return (self.weigths * f_values).sum()
        

    def learn(self):
        '''
        
        '''
        ...

